In [9]:
import helper_functions
import numpy as np
import keras
import pickle
from keras.optimizers import RMSprop
# CNN
from keras.models import Sequential  
from keras.layers import Dense, Conv1D, Activation, Conv2D, MaxPooling2D, Input
from keras.layers import InputLayer, Dropout, LSTM, BatchNormalization, Flatten, ZeroPadding1D, AveragePooling2D, BatchNormalization, MaxPooling1D, AveragePooling1D, GlobalMaxPooling1D

#import helper_functions
from keras.layers import GlobalMaxPooling1D, Dropout, BatchNormalization


ImportError: cannot import name 'RMSprop' from 'keras.optimizers' (c:\users\martín\appdata\local\programs\python\python39\lib\site-packages\keras\optimizers.py)

In [2]:
data_path = 'data'
with open(data_path, 'rb') as f:
    data = pickle.Unpickler(f).load()

In [3]:
# Shuffle the data
dataToShuffle = []

for log, mfcc, label in zip(data['log-mel'],data['mfccs'],data['classes']):
    dataToShuffle.append([log,mfcc,label])

ShuffledData = np.array(dataToShuffle)
np.random.shuffle(ShuffledData)

In [4]:
# Separate the data into training and test datasets
Train = ShuffledData[:45305]
Test = ShuffledData[45305:]

log_train = []
mfcc_train = []
y_train = []

log_test = []
mfcc_test = []
y_test = []

for log, mfcc, label in Train:
    log_train.append(log)
    mfcc_train.append(mfcc)
    y_train.append(label)
    
for log, mfcc, label in Test:
    log_test.append(log)
    mfcc_test.append(mfcc)
    y_test.append(label)


In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

for i in range(2):
    label = y_train[45000+i]
    plt.figure()
    plt.title("Class id: "+ str(label))
    plt.imshow(x_train[45000+i])

In [56]:
log_test[0].shape


(15, 173)

In [54]:
InputShape = log_test[0].shape
output_dim = y_test[0].shape[0]

dropout = 0.2

#based on model 1 from https://www.researchgate.net/publication/348432098_Speech_recognition_based_on_Convolutional_neural_networks_and_MFCC_algorithm
# Creation of the CNN
model = Sequential()
model.add(InputLayer(input_shape=InputShape))
model.add(ZeroPadding1D(padding=2))
model.add(Conv1D(filters=256, kernel_size=10))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Flatten())
#model.add(Conv1D(filters=256, kernel_size=5))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))
#model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Dense(output_dim))
model.add(Activation('softmax'))

opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

ValueError: Could not interpret optimizer identifier: <tensorflow.python.keras.optimizer_v2.rmsprop.RMSprop object at 0x000002BB51751CD0>

In [5]:
#based on https://towardsdatascience.com/tensorflow-speech-recognition-challenge-solution-outline-9c42dbd219c9
#this one could be quite good, ^^ was getting like 80-85% accuracy (It's the 1D CNN)
model = Sequential()

model.add(BatchNormalization())

model.add(Conv1D(64,kernel_size=2,padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64,kernel_size=2,padding='same'))
model.add(Activation('relu'))

model.add(Dropout(0.2))
model.add(MaxPooling1D(2))

model.add(Conv1D(128, 3,padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 3,padding='same'))
model.add(Activation('relu'))

model.add(Dropout(0.2))
model.add(MaxPooling1D(2))

model.add(Conv1D(128, 3,padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 3,padding='same'))
model.add(Activation('relu'))

model.add(Dropout(0.2))
model.add(MaxPooling1D(2))

model.add(Conv1D(128, 3,padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 3,padding='same'))
model.add(Activation('relu'))


model.add(Dropout(0.2))
model.add(MaxPooling1D(2))

model.add(GlobalMaxPooling1D())

model.add(Dense(12, activation="softmax"))
    

In [6]:

opt = RMSprop(lr=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

ValueError: Could not interpret optimizer identifier: <tensorflow.python.keras.optimizer_v2.rmsprop.RMSprop object at 0x00000218D7696670>

In [ ]:
#Train the model 
x_train = np.array(mfcc_train)
y_train = np.array(y_train)
model.fit(x=x_train, y=y_train, epochs=12, batch_size=128)

In [ ]:
#anontherone from https://medium.com/x8-the-ai-community/audio-classification-using-cnn-coding-example-f9cbd272269e
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))
#Compile
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.adam(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
#Train The Model
x_train = np.array(mfcc_train)
y_train = np.array(y_train)
model.fit(x_train, y_train, batch_size=4, epochs=10, verbose=1, validation_data=(x_test, y_test))

In [ ]:
# Testing the model 
x_test = np.array(mfcc_test)
y_test = np.array(y_test)
score = model.evaluate(x_test,y_test)